# _Notebook Jupyter_ 1a_trataCSVsSiscori

# Importação e tratamento dos dados Siscori

Os dados importados no Siscore vêm em arquivos do tipo CSV no formato <b>CAPINNAAMM</b>, onde NN é o número do capítulo extraído, AA é o ano com dois dígitos e MM é o mês com dois dígitos, formando o período de refência dos dados extraídos. O arquivo CSV obtido vem configurado com o separador "<b>@</b>" e descrição da coluna com excesso de espaços, o que precisa de uma camada de tratamento para correta importação dos dados.

### Importação das Bibliotecas

In [1]:
import pandas as pd
import numpy as np
import os, time

In [2]:
# Data e hora da execução do script
print(f'Código executado em {time.strftime("%d/%m/%Y às %H:%M", time.localtime(time.time()))}')

Código executado em 12/01/2022 às 18:22


### Definindo dados gerais

Cria lista de arquivos CSV contidos na pasta atual

In [3]:
arqsCSV = []
for arquivo in os.listdir("./bases/siscori/"):
    if arquivo[-3:].upper()=="CSV" and arquivo[:4]=='CAPI':
        arqsCSV.append("./bases/siscori/"+arquivo)
arqsCSV=sorted(arqsCSV)
print(arqsCSV)

['./bases/siscori/CAPI872001.CSV', './bases/siscori/CAPI872002.CSV', './bases/siscori/CAPI872003.CSV', './bases/siscori/CAPI872004.CSV', './bases/siscori/CAPI872005.CSV', './bases/siscori/CAPI872006.CSV', './bases/siscori/CAPI872007.CSV', './bases/siscori/CAPI872008.CSV', './bases/siscori/CAPI872009.CSV', './bases/siscori/CAPI872010.CSV', './bases/siscori/CAPI872011.CSV', './bases/siscori/CAPI872012.CSV', './bases/siscori/CAPI872101.CSV', './bases/siscori/CAPI872102.CSV', './bases/siscori/CAPI872103.CSV', './bases/siscori/CAPI872104.CSV', './bases/siscori/CAPI872105.CSV', './bases/siscori/CAPI872106.CSV']


Cria variáveis com nomes das colunas e seus tipos

In [4]:
tipos = {'NUMERO DE ORDEM': str,
         'ANOMES': str,
         'COD.NCM': str,
         'DESCRICAO DO CODIGO NCM': object,
         'PAIS.OR': int,
         'PAIS DE ORIGEM': object,
         'PAIS.AQ': int,
         'PAIS DE AQUISICAO': object,
         'UND.ESTAT.': int,
         'UNIDADE DE MEDIDA': object,
         'UNIDADE COMERC.': object,
         'DESCRICAO DO PRODUTO': object,
         'QTDE ESTATISTICA': float,
         'PESO LIQUIDO': float,
         'VMLE DOLAR': float,
         'VL FRETE DOLAR': float,
         'VL SEGURO DOLAR': float,
         'VALOR UN.PROD.DOLAR': float,
         'QTD COMERCIAL.': float,
         'TOT.UN.PROD.DOLAR': float,
         'UNIDADE DESEMBARQUE': object,
         'UNIDADE DESEMBARACO': object,
         'INCOTERM': object,
         'NAT.INFORMACAO': object,
         'SITUACAO DO DESPACHO': object}
colunas = list(tipos.keys())

Inicializa a variável que conterá o tamanho total do dataset original

In [5]:
tamanhoDataset=0

Inicializa um dataframe vazio que conterá os dados finais

In [6]:
df = pd.DataFrame(columns = colunas)
df.head()

,NUMERO DE ORDEM,ANOMES,COD.NCM,DESCRICAO DO CODIGO NCM,PAIS.OR,PAIS DE ORIGEM,PAIS.AQ,PAIS DE AQUISICAO,UND.ESTAT.,UNIDADE DE MEDIDA,...,VL FRETE DOLAR,VL SEGURO DOLAR,VALOR UN.PROD.DOLAR,QTD COMERCIAL.,TOT.UN.PROD.DOLAR,UNIDADE DESEMBARQUE,UNIDADE DESEMBARACO,INCOTERM,NAT.INFORMACAO,SITUACAO DO DESPACHO


### Importa cada CSV, trata e concatena no DataFrame final

In [7]:
# Executa para cada CSV na lista
for arqCSV in arqsCSV:
    print('Iniciando ' + arqCSV + '.')
    dftemp = pd.read_csv(arqCSV,
                         sep='@', 
                         decimal=r',', 
                         engine='python', 
                         encoding = "ISO-8859-1",
                         header = 0,
                         names = colunas,
                         dtype = tipos, 
                         quotechar="'", 
                         error_bad_lines=False)
    print('DataFrame carregado...\nAplicando filtros...')
    # Elimina os registros sem valores ou nulos
    dftemp = dftemp.dropna()
    # Incrementa o tamanho do Dataset
    tamanhoDataset += dftemp[dftemp.columns[0]].count()
    print('Quantidade de registros válidos: ' + str(tamanhoDataset))
    # Filtra o DataFrame somente com os registros de interesse
    # Filtro 1: NCM de interesse: 87141000
    indiceNCM = dftemp['COD.NCM'] == '87141000'
    dftemp = dftemp[indiceNCM]
    # Filtro 2: Descrição contendo palavras da lista
    listafiltro = ["transm", "corrente", "coroa", "pinhao|pinhão"] # A barra vertical (|) faz o "ou".
    for termo in listafiltro:
        dftemp=dftemp[dftemp['DESCRICAO DO PRODUTO'].str.contains(termo, case=False)]
    # Concatena com o DataFrame final
    df = pd.concat([df,dftemp])
    print(arqCSV + ' finalizado.\n')

Iniciando ./bases/siscori/CAPI872001.CSV.


Skipping line 390699: field larger than field limit (131072)


DataFrame carregado...
Aplicando filtros...
Quantidade de registros válidos: 491150
./bases/siscori/CAPI872001.CSV finalizado.

Iniciando ./bases/siscori/CAPI872002.CSV.
DataFrame carregado...
Aplicando filtros...
Quantidade de registros válidos: 867647
./bases/siscori/CAPI872002.CSV finalizado.

Iniciando ./bases/siscori/CAPI872003.CSV.
DataFrame carregado...
Aplicando filtros...
Quantidade de registros válidos: 1303525
./bases/siscori/CAPI872003.CSV finalizado.

Iniciando ./bases/siscori/CAPI872004.CSV.
DataFrame carregado...
Aplicando filtros...
Quantidade de registros válidos: 1633743
./bases/siscori/CAPI872004.CSV finalizado.

Iniciando ./bases/siscori/CAPI872005.CSV.
DataFrame carregado...
Aplicando filtros...
Quantidade de registros válidos: 1881276
./bases/siscori/CAPI872005.CSV finalizado.

Iniciando ./bases/siscori/CAPI872006.CSV.
DataFrame carregado...
Aplicando filtros...
Quantidade de registros válidos: 2111936
./bases/siscori/CAPI872006.CSV finalizado.

Iniciando ./bases/

Skipping line 205695: '@' expected after '''
Skipping line 205941: field larger than field limit (131072)
Skipping line 328937: '@' expected after '''
Skipping line 333250: '@' expected after '''
Skipping line 333251: '@' expected after '''


DataFrame carregado...
Aplicando filtros...
Quantidade de registros válidos: 3092950
./bases/siscori/CAPI872009.CSV finalizado.

Iniciando ./bases/siscori/CAPI872010.CSV.


...

DataFrame carregado...
Aplicando filtros...
Quantidade de registros válidos: 6481393
./bases/siscori/CAPI872104.CSV finalizado.

Iniciando ./bases/siscori/CAPI872105.CSV.


Skipping line 1432: '@' expected after '''
Skipping line 1433: field larger than field limit (131072)
Skipping line 6175: '@' expected after '''
Skipping line 277564: '@' expected after '''
Skipping line 278426: field larger than field limit (131072)
Skipping line 280731: '@' expected after '''
Skipping line 281441: field larger than field limit (131072)
Skipping line 282315: '@' expected after '''
Skipping line 282700: field larger than field limit (131072)
Skipping line 333245: '@' expected after '''
Skipping line 386253: field larger than field limit (131072)


DataFrame carregado...
Aplicando filtros...
Quantidade de registros válidos: 6963712
./bases/siscori/CAPI872105.CSV finalizado.

Iniciando ./bases/siscori/CAPI872106.CSV.


Skipping line 8619: field larger than field limit (131072)
Skipping line 9337: field larger than field limit (131072)
Skipping line 10237: field larger than field limit (131072)


DataFrame carregado...
Aplicando filtros...
Quantidade de registros válidos: 7693634
./bases/siscori/CAPI872106.CSV finalizado.



### Resetando o índice do DataFrame importado

In [8]:
df.reset_index(inplace=True, drop=True)

### Verificando o DataFrame importado

In [9]:
df.sample(3)

,NUMERO DE ORDEM,ANOMES,COD.NCM,DESCRICAO DO CODIGO NCM,PAIS.OR,PAIS DE ORIGEM,PAIS.AQ,PAIS DE AQUISICAO,UND.ESTAT.,UNIDADE DE MEDIDA,...,VL FRETE DOLAR,VL SEGURO DOLAR,VALOR UN.PROD.DOLAR,QTD COMERCIAL.,TOT.UN.PROD.DOLAR,UNIDADE DESEMBARQUE,UNIDADE DESEMBARACO,INCOTERM,NAT.INFORMACAO,SITUACAO DO DESPACHO
9388,0000000451880000100005,202011,87141000,PARTES ACESSÓRIOS P/MOTOCICLET,160,"CHINA, REPUBLICA POP",160,"CHINA, REPUBLICA POP",10,QUILOGRAMA LIQUIDO,...,2772.11,333.66,1.42000,300.0,426.00,N/INFORMADO,ITAJAI,FOB,EFETIVA,DI DESEMBARAÇADA
17356,0000001935400000400023,202104,87141000,PARTES ACESSÓRIOS P/MOTOCICLET,160,"CHINA, REPUBLICA POP",767,SUICA,10,QUILOGRAMA LIQUIDO,...,8427.66,263.89,5.87000,40.0,234.80,N/INFORMADO,PORTO DE VITORIA,FOB,EFETIVA,DI DESEMBARAÇADA
19149,0000001790740000100010,202106,87141000,PARTES ACESSÓRIOS P/MOTOCICLET,160,"CHINA, REPUBLICA POP",351,HONG KONG,10,QUILOGRAMA LIQUIDO,...,4485.00,232.35,4.62885,200.0,925.77,N/INFORMADO,ITAJAI,FOB,EFETIVA,DI DESEMBARAÇADA


In [10]:
df.shape

(19317, 25)

### Remover espaços em excesso nos campos string

In [11]:
# Remove espaços em excesso das colunas em colstr
colstr = ['DESCRICAO DO CODIGO NCM',
          'PAIS DE ORIGEM',
          'PAIS DE AQUISICAO',
          'UNIDADE DE MEDIDA',
          'UNIDADE COMERC.',
          'DESCRICAO DO PRODUTO',
          'UNIDADE DESEMBARQUE',
          'UNIDADE DESEMBARACO',
          'INCOTERM',
          'NAT.INFORMACAO',
          'SITUACAO DO DESPACHO']
for coluna in colstr:
    df[coluna]=df[coluna].str.strip()

In [12]:
df['DESCRICAO DO PRODUTO'][5]

'80348 KIT DE TRANSMISSÃO, COMPOSTO DE CORRENTE, COROA E PINHÃO PARA MOTOCICLETA CBX 250 TWISTER, MARCA ALLEN.'

### Excluindo colunas desnecessárias

In [13]:
excluir=['NUMERO DE ORDEM',
         'ANOMES',
         'COD.NCM',
         'DESCRICAO DO CODIGO NCM',
         'PAIS.OR',
         'PAIS.AQ',
         'PAIS DE AQUISICAO',
         'UND.ESTAT.',
         'UNIDADE DE MEDIDA',
         'UNIDADE COMERC.',
         'QTDE ESTATISTICA',
         'PESO LIQUIDO',
         'VMLE DOLAR',
         'VL FRETE DOLAR',
         'VL SEGURO DOLAR',
         'QTD COMERCIAL.',
         'TOT.UN.PROD.DOLAR',
         'UNIDADE DESEMBARQUE',
         'UNIDADE DESEMBARACO',
         'INCOTERM',
         'NAT.INFORMACAO',
         'SITUACAO DO DESPACHO']

In [14]:
df=df.drop(excluir, axis=1)

In [15]:
df.head(3)

,PAIS DE ORIGEM,DESCRICAO DO PRODUTO,VALOR UN.PROD.DOLAR
0,"CHINA, REPUBLICA POP",007293# KIT TRANSMISSAO STANDARD TEMPERADO COM...,3.728
1,"CHINA, REPUBLICA POP",007295# KIT TRANSMISSAO STANDARD TEMPERADO COM...,3.700
2,"CHINA, REPUBLICA POP",007296# KIT TRANSMISSAO STANDARD TEMPERADO COM...,3.686


In [16]:
df.tail(3)

,PAIS DE ORIGEM,DESCRICAO DO PRODUTO,VALOR UN.PROD.DOLAR
19314,"CHINA, REPUBLICA POP",KIT DE TRANSMISSÃO AÇO TITAN 99 1045 COMPOSTO ...,3.581
19315,"CHINA, REPUBLICA POP","KIT DE TRANSMISSÃO AÇO YBR 125 00/02 1045, COM...",3.450
19316,"CHINA, REPUBLICA POP","KIT DE TRANSMISSÃO AÇO YBR 125 03/05, COMPOSTO...",3.503


### Exportando o DataFrame

Exportando para um arquivo CSV

In [17]:
df.to_csv(r'./bases/dataframe.csv', index = False, header = True)

Exportando para um arquivo de planilha do Excel

In [18]:
df.to_excel(r'./bases/dataframe.xlsx', index = False, header = True)

Compara o tamanho total do Dataset inicial e final

In [19]:
print('Qtd de registros Dataset original: ' + str(tamanhoDataset))
print('Qtd de registros Dataset final:    ' + str(df[df.columns[0]].count()))

Qtd de registros Dataset original: 7693634
Qtd de registros Dataset final:    19317
